In [1]:
import torch
import pywt
import pandas               as  pd
import numpy                as  np
import matplotlib.pyplot    as  plt
import torch.nn             as  nn

# Hyper Parameters

In [2]:
# Dataset
samples_per_test    = 32768
Total_test          = 29
test_Duration       = 2

# model         Maybe adding dropout in LSTM is good
input_horizon       = 200
# model-LSTM
LSTM_outFeature     = 100
LSTM_NumLayer       = 2
LSTM_hidden_size    = 64

# CWT
Scales              = 50
wavelet_lis         = ['mexh','morl','gaus1','cmor1-1.0']   # Define the wavelet and scales
wavelet             = wavelet_lis[1]                        # You can choose other wavelet functions as well
Coefficient_Real    = True



# Reading dataset
Read the CSV file into a DataFrame

In [3]:
df = pd.read_csv('Dataset_BinaryClass.csv')
df['Faulty'] = df['Faulty'].replace({False: 0, True: 1})

"""
print(df.head())

its better to keep numpy format because CWT uses numpy and before network make it pytorch

X = torch.tensor(df['Datas' ].values).reshape(Total_test,samples_per_test)
Y = torch.tensor(df['Faulty'].values).reshape(Total_test,samples_per_test)
"""

X_numpy = df['Datas' ].values.reshape(Total_test,samples_per_test)
Y_numpy = df['Faulty'].values.reshape(Total_test,samples_per_test)

In [30]:
df

,Datas,Faulty
0,0.000000,0
1,0.002565,0
2,0.000000,0
3,0.000000,0
4,0.000000,0
...,...,...
950267,0.102588,0
950268,0.087200,0
950269,0.017953,0
950270,-0.164141,0


# Continuous Wavelet transform (CWT)

In [29]:
scales = np.arange(1, Scales)

for wavelet in reversed(wavelet_lis):
    # Perform Continuous Wavelet Transform
    print(wavelet)
    coefficients, frequencies = pywt.cwt(X_numpy[1,:input_horizon], scales, wavelet)

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.imshow(np.abs(coefficients), extent=[0, 1, 1, Scales], aspect='auto', cmap='jet')
    plt.colorbar(label='Magnitude')
    plt.title('Continuous Wavelet Transform')
    plt.xlabel('Time')
    plt.ylabel('Scale')
    plt.show()

if Coefficient_Real:
    coefficients = np.abs(coefficients)

cmor1-1.0
gaus1
morl
mexh


In [26]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self,
                input_horizon   :int ,
                hidden_size     :int    = LSTM_hidden_size,
                num_layers      :int    = LSTM_NumLayer,
                output_size     :int    = LSTM_outFeature,
                DropOut         :float  = 0)->None:
        super(LSTMModel, self).__init__()
        self.hidden_size    = hidden_size
        self.num_layers     = num_layers
        
        self.lstm           = nn.LSTM(input_horizon, hidden_size, num_layers, batch_first=True)
        self.fc             = nn.Linear(hidden_size, output_size)
        
    def forward(self, x)->tuple:
        out, (h_state,C_state) = self.lstm(x)
        
        print(out.shape)
        if len(out.shape)==2:
            out = self.fc(out[:, :])
        if len(out.shape)==3:
            out = self.fc(out[:, -1, :])
        return out#,h_state,C_state

in ``` h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) ``` x is input and data will be generated in device

In [27]:
model = LSTMModel(input_horizon)


# coefficients

output = model(torch.tensor(X_numpy[1,:input_horizon], dtype=torch.float32).unsqueeze(0))

# print("Output shape:", output.shape)


torch.Size([1, 64])
